In [2]:
from sklearn.model_selection import train_test_split
import onnxruntime as rt
import onnx
from skl2onnx.common.data_types import FloatTensorType
from skl2onnx import to_onnx
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.pipeline import Pipeline
from skl2onnx import convert_sklearn
from sklearn.preprocessing import StandardScaler
# define a XGBoost classifier
import xgboost as xgb
import warnings
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.base import BaseEstimator, TransformerMixin
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import GradientBoostingClassifier
from aif360.sklearn.datasets import fetch_compas
from aif360.sklearn.metrics import disparate_impact_ratio, consistency_score, generalized_entropy_error
from aif360.sklearn.detectors import bias_scan
from aif360.sklearn.inprocessing import AdversarialDebiasing
from aif360.datasets import BinaryLabelDataset, StandardDataset
from aif360.metrics import BinaryLabelDatasetMetric
from xgboost import XGBClassifier
from skl2onnx import update_registered_converter
from skl2onnx.common._apply_operation import apply_identity
from skl2onnx.common.shape_calculator import (
    calculate_linear_classifier_output_shapes,
)
from skl2onnx import update_registered_converter
from onnxmltools.convert.xgboost.operator_converters.XGBoost import convert_xgboost
warnings.filterwarnings("ignore")  # Ignore runtime warnings
# Temporarily adjust pandas display settings for large DataFrames
pd.set_option('display.max_rows', 100)  # Ensure 100 rows can be displayed
pd.set_option('display.max_columns', None)  # Ensure all columns can be displayed
pd.set_option('display.width', None)  # Automatically adjust display width to terminal size
pd.set_option('display.max_colwidth', None)  # Ensure full width of column content is shown
pd.set_option('display.float_format', '{:.4f}'.format)  # Format the float numbers for better readability


pip install 'aif360[Reductions]'
pip install 'aif360[Reductions]'
pip install 'aif360[inFairness]'
pip install 'aif360[Reductions]'
pip install 'aif360[OptimalTransport]'
pip install 'aif360[FACTS]'


In [11]:
data = pd.read_csv('../data/investigation_train_large_checked_adjusted3.csv')

In [12]:
# Step 2: Define the target and features
y = data['checked']  # Assuming 'checked' is the target column
X = data.drop(['checked', 'Ja', 'Nee'], axis=1)

# Ensure features are float32
X = X.astype(np.float32)

# Step 3: Split the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, shuffle=True, stratify=y
)

In [13]:
testing_session = rt.InferenceSession("./received_folder/model_1.onnx")

In [14]:
def calculate_prediction_consistency(testing_session, X_test, feature_name, values):
    """
    Calculate the fraction of cases where predictions are the same for different values of a given feature.

    Parameters:
        testing_session (InferenceSession): The ONNX model inference session.
        X_test (pd.DataFrame): Test dataset.
        feature_name (str): The feature to modify.
        values (list): List of two values to compare for the feature.

    Returns:
        float: Fraction of cases where predictions are the same.
    """
    same_predictions_count = 0
    total_samples = len(X_test)

    # Get the input name expected by the ONNX model
    input_name = testing_session.get_inputs()[0].name

    for _, row in X_test.iterrows():
        # Convert the row to a DataFrame with one row
        X_sample = pd.DataFrame([row])

        # Modify the feature for the first value
        X_sample_1 = X_sample.copy()
        X_sample_1[feature_name] = values[0]
        y_pred_1 = testing_session.run(None, {input_name: X_sample_1.values.astype(np.float32)})[0]

        # Modify the feature for the second value
        X_sample_2 = X_sample.copy()
        X_sample_2[feature_name] = values[1]
        y_pred_2 = testing_session.run(None, {input_name: X_sample_2.values.astype(np.float32)})[0]

        # Check if predictions are the same
        if np.array_equal(y_pred_1, y_pred_2):
            same_predictions_count += 1

    # Calculate the fraction of cases where predictions are the same
    return same_predictions_count / total_samples


In [15]:
age_fraction = calculate_prediction_consistency(
    testing_session=testing_session,
    X_test=X_test,
    feature_name='persoon_leeftijd_bij_onderzoek',
    values=[25, 40]
)
print("Fraction of cases where predictions are the same for age 25 and 40:", age_fraction)


gender_fraction = calculate_prediction_consistency(
    testing_session=testing_session,
    X_test=X_test,
    feature_name='persoon_geslacht_vrouw',
    values=[0.0, 1.0]
)
print("Fraction of cases where predictions are the same for men and women:", gender_fraction)

language_fraction = calculate_prediction_consistency(
    testing_session=testing_session,
    X_test=X_test,
    feature_name='persoonlijke_eigenschappen_taaleis_voldaan',
    values=[0.0, 1.0]
)
print("Fraction of cases where predictions are the same for dutch speakers and non-dutch speakers:", language_fraction)

children_fraction = calculate_prediction_consistency(
    testing_session=testing_session,
    X_test=X_test,
    feature_name='relatie_kind_heeft_kinderen',
    values=[0.0, 1.0]
)
print("Fraction of cases where predictions are the same for people with or without children:", children_fraction)


Fraction of cases where predictions are the same for age 25 and 40: 1.0
Fraction of cases where predictions are the same for men and women: 0.976736899819848
Fraction of cases where predictions are the same for dutch speakers and non-dutch speakers: 0.9680034463852119
Fraction of cases where predictions are the same for people with or without children: 1.0


In [16]:
def test_model_for_groups(pipe, x_training, y_training, x_testing, y_testing, group_dicts):
    """
    Tests the model and calculates fairness metrics for multiple privileged/unprivileged group definitions.
    
    Args:
        pipe: The model pipeline to test.
        x_training: Training features.
        y_training: Training labels.
        x_testing: Testing features.
        y_testing: Testing labels.
        group_dicts: List of dictionaries defining privileged and unprivileged groups.
    
    Returns:
        pd.DataFrame: Dataframe with calculated metrics for each privileged/unprivileged group combination.
    """
    def intoBinary(data, attribute, split):
        """
        Converts a continuous attribute into binary based on a percentile split.
        """
        sorted_data = data.sort_values(by=attribute)
        split_index = int(len(sorted_data) * split)
        split_threshold = sorted_data.iloc[split_index][attribute]
        print(f"Boundary for {attribute}: {split_threshold}")
        binary_attribute = pd.Series(0, index=data.index)
        binary_attribute[sorted_data[attribute] > split_threshold] = 1
        data[attribute] = binary_attribute
        return data

    if isinstance(pipe, rt.InferenceSession):  # ONNX model
        print("Using ONNX model for predictions")
        predictions = pipe.run(None, {'X': x_testing.values.astype(np.float32)})
        y_pred = predictions[0].round().astype(int)  # Ensure predictions are integer class labels
    else:  # Assume scikit-learn pipeline
        print("Training the model")
        pipe.fit(x_training, y_training)
        y_pred = pipe.predict(x_testing)

    # Prepare a DataFrame with features and predictions
    data_frame = pd.concat([x_testing.reset_index(drop=True), pd.DataFrame(y_pred, columns=['checked'])], axis=1)

    results = []

    for group in group_dicts:
        protected_attr = group['protected_attribute']
        privileged_groups = group['privileged']
        unprivileged_groups = group['unprivileged']
        
        # Convert the protected attribute to binary
        data_frame = intoBinary(data_frame, protected_attr, split=0.1)  # Using 0.1 as an example split value

        # BinaryLabelDataset creation
        bld = BinaryLabelDataset(
            df=data_frame,
            favorable_label=0,
            unfavorable_label=1,
            label_names=['checked'],
            unprivileged_protected_attributes=[[group['unprivileged_value']]],
            privileged_protected_attributes=[[group['privileged_value']]],
            protected_attribute_names=[protected_attr]
        )
        # Count outcomes for privileged and unprivileged groups
        privileged_indices = bld.protected_attributes[:, 0] == group['privileged_value']
        unprivileged_indices = bld.protected_attributes[:, 0] == group['unprivileged_value']

        privileged_labels = bld.labels[privileged_indices]
        unprivileged_labels = bld.labels[unprivileged_indices]

        privileged_checked_count = (privileged_labels == bld.favorable_label).sum()
        privileged_unchecked_count = (privileged_labels == bld.unfavorable_label).sum()

        unprivileged_checked_count = (unprivileged_labels == bld.favorable_label).sum()
        unprivileged_unchecked_count = (unprivileged_labels == bld.unfavorable_label).sum()

        # Print the counts
        print(f"\nCounts for Protected Attribute '{protected_attr}':")
        print(f"Privileged Group ({privileged_groups}):")
        print(f"  Checked (Favorable Outcomes): {privileged_checked_count}")
        print(f"  Unchecked (Unfavorable Outcomes): {privileged_unchecked_count}")
        print(f"Unprivileged Group ({unprivileged_groups}):")
        print(f"  Checked (Favorable Outcomes): {unprivileged_checked_count}")
        print(f"  Unchecked (Unfavorable Outcomes): {unprivileged_unchecked_count}")
        # Validate the dataset
        bld.validate_dataset()

        # Metric calculation
        metric = BinaryLabelDatasetMetric(bld, privileged_groups=privileged_groups, unprivileged_groups=unprivileged_groups)


        
        # Metrics
        disparate_impact = metric.disparate_impact()
        stat_par_diff = metric.statistical_parity_difference()
        mean_difference = metric.mean_difference()
        consistency = metric.consistency(n_neighbors=3)

        # Print metrics
        print(f"\nMetrics for protected attribute '{protected_attr}':")
        print(f"Privileged Groups: {privileged_groups}")
        print(f"Unprivileged Groups: {unprivileged_groups}")
        print(f"Disparate Impact: {disparate_impact}")
        print(f"Statistical Parity Difference: {stat_par_diff}")
        print(f"Mean Difference: {mean_difference}")
        print(f"Consistency: {consistency}")

        # Store the results
        results.append({
            'Protected Attribute': protected_attr,
            'Privileged Groups': privileged_groups,
            'Unprivileged Groups': unprivileged_groups,
            'Disparate Impact': disparate_impact,
            'Statistical Parity Difference': stat_par_diff,
            'Mean Difference': mean_difference,
            'Consistency': consistency
        })

    # Convert results to a DataFrame
    results_df = pd.DataFrame(results)
    print("\nFairness Metrics Summary:")
    print(results_df)

    return results_df


In [17]:
group_dicts = [
    {
        'protected_attribute': 'persoon_leeftijd_bij_onderzoek',
        'privileged': [{'persoon_leeftijd_bij_onderzoek': 1}],
        'unprivileged': [{'persoon_leeftijd_bij_onderzoek': 0}],
        'privileged_value': 1,
        'unprivileged_value': 0
    },
    {
        'protected_attribute': 'persoon_geslacht_vrouw',
        'privileged': [{'persoon_geslacht_vrouw': 0}],
        'unprivileged': [{'persoon_geslacht_vrouw': 1}],
        'privileged_value': 0,
        'unprivileged_value': 1
    },
    {
        'protected_attribute': 'relatie_overig_kostendeler',
        'privileged': [{'relatie_overig_kostendeler': 0}],
        'unprivileged': [{'relatie_overig_kostendeler': 1}],
        'privileged_value': 0,
        'unprivileged_value': 1
    }
]
results_df = test_model_for_groups(
    pipe=testing_session, 
    x_training=X_train, 
    y_training=y_train, 
    x_testing=X_test, 
    y_testing=y_test, 
    group_dicts=group_dicts
)

Using ONNX model for predictions
Boundary for persoon_leeftijd_bij_onderzoek: 36.0

Counts for Protected Attribute 'persoon_leeftijd_bij_onderzoek':
Privileged Group ([{'persoon_leeftijd_bij_onderzoek': 1}]):
  Checked (Favorable Outcomes): 18256
  Unchecked (Unfavorable Outcomes): 4366
Unprivileged Group ([{'persoon_leeftijd_bij_onderzoek': 0}]):
  Checked (Favorable Outcomes): 1941
  Unchecked (Unfavorable Outcomes): 971

Metrics for protected attribute 'persoon_leeftijd_bij_onderzoek':
Privileged Groups: [{'persoon_leeftijd_bij_onderzoek': 1}]
Unprivileged Groups: [{'persoon_leeftijd_bij_onderzoek': 0}]
Disparate Impact: 0.8259609891915709
Statistical Parity Difference: -0.14044983561659807
Mean Difference: -0.14044983561659807
Consistency: [0.788452]
Boundary for persoon_geslacht_vrouw: 0.0

Counts for Protected Attribute 'persoon_geslacht_vrouw':
Privileged Group ([{'persoon_geslacht_vrouw': 0}]):
  Checked (Favorable Outcomes): 10352
  Unchecked (Unfavorable Outcomes): 2802
Unpri

In [18]:
def evaluate_model_by_group(testing_session, X_test, y_test, group_column, group_mapping=None, bins=None, labels=None):
    """
    Evaluate model performance across demographic groups.

    Parameters:
        testing_session (InferenceSession): The ONNX model inference session.
        X_test (pd.DataFrame): Test dataset features.
        y_test (pd.Series): True labels for the test dataset.
        group_column (str): Column used to define groups.
        group_mapping (dict): Mapping of values in `group_column` to group labels.
        bins (list): Bin edges for numeric grouping.
        labels (list): Labels for bins if numeric grouping is used.

    Returns:
        dict: A dictionary containing performance metrics for each group.
    """
    # Copy test data to avoid modification
    X_test_grouped = X_test.copy()

    # Handle group definitions (mapping or binning)
    if group_mapping:
        X_test_grouped['group'] = X_test_grouped[group_column].map(group_mapping)
    elif bins and labels:
        X_test_grouped['group'] = pd.cut(X_test_grouped[group_column], bins=bins, labels=labels)
    else:
        raise ValueError("Provide either `group_mapping` or `bins` and `labels` to define groups.")

    # Drop rows with NaN in the 'group' column
    X_test_grouped = X_test_grouped.dropna(subset=['group'])

    # Align y_test with X_test_grouped
    y_test_aligned = y_test.loc[X_test_grouped.index]

    # Debug: Check group distribution
    print("Group distribution:\n", X_test_grouped['group'].value_counts())

    # Initialize dictionary to store results
    results = {}

    # Evaluate performance for each group
    for group in X_test_grouped['group'].unique():
        # Filter data for the current group
        X_group = X_test_grouped[X_test_grouped['group'] == group].drop(columns=['group'])
        y_group = y_test_aligned[X_test_grouped['group'] == group]

        # Predict using the model
        y_pred_group = testing_session.run(None, {'X': X_group.values.astype(np.float32)})[0]

        # Calculate evaluation metrics
        accuracy_group = accuracy_score(y_group, y_pred_group)
        precision_group = precision_score(y_group, y_pred_group, zero_division=0)
        recall_group = recall_score(y_group, y_pred_group, zero_division=0)
        f1_score_group = f1_score(y_group, y_pred_group, zero_division=0)

        # Store metrics for the group
        results[group] = {
            'Accuracy': accuracy_group,
            'Precision': precision_group,
            'Recall': recall_group,
            'F1 Score': f1_score_group
        }

    return results

In [19]:
age_bins = [0, 30, 40, 50, 60, 120]
age_labels = ['young_adult', 'youngish_adult', 'middle_aged_adult', 'older_adult', 'senior']

age_results = evaluate_model_by_group(
    testing_session=testing_session,
    X_test=X_test,
    y_test=y_test,
    group_column='persoon_leeftijd_bij_onderzoek',
    bins=age_bins,
    labels=age_labels
)

print("Age Group Results:")
for group, metrics in age_results.items():
    print(f"{group}: {metrics}")


gender_mapping = {1.0: 'woman', 0.0: 'man'}

gender_results = evaluate_model_by_group(
    testing_session=testing_session,
    X_test=X_test,
    y_test=y_test,
    group_column='persoon_geslacht_vrouw',
    group_mapping=gender_mapping
)

print("Gender Results:")
for group, metrics in gender_results.items():
    print(f"{group}: {metrics}")

language_mapping = {1.0: 'other_language', 0.0: 'not'}

language_results = evaluate_model_by_group(
    testing_session=testing_session,
    X_test=X_test,
    y_test=y_test,
    group_column='persoonlijke_eigenschappen_taaleis_voldaan',
    group_mapping=language_mapping
)

print("Language Results:")
for group, metrics in language_results.items():
    print(f"{group}: {metrics}")

children_mapping = {1.0: 'has_child', 0.0: 'not'}

children_results = evaluate_model_by_group(
    testing_session=testing_session,
    X_test=X_test,
    y_test=y_test,
    group_column='relatie_kind_heeft_kinderen',
    group_mapping=children_mapping
)

print("Children Results:")
for group, metrics in children_results.items():
    print(f"{group}: {metrics}")

Group distribution:
 group
older_adult          8695
middle_aged_adult    8234
youngish_adult       4013
senior               3582
young_adult          1010
Name: count, dtype: int64
Age Group Results:
senior: {'Accuracy': 0.907035175879397, 'Precision': 0.5656877897990726, 'Recall': 0.8755980861244019, 'F1 Score': 0.6873239436619718}
older_adult: {'Accuracy': 0.908223116733755, 'Precision': 0.37479935794542535, 'Recall': 0.9609053497942387, 'F1 Score': 0.5392609699769053}
young_adult: {'Accuracy': 0.902970297029703, 'Precision': 0.8073089700996677, 'Recall': 0.8586572438162544, 'F1 Score': 0.8321917808219178}
youngish_adult: {'Accuracy': 0.8888612010964366, 'Precision': 0.7456472369417109, 'Recall': 0.8995433789954338, 'F1 Score': 0.8153973509933775}
middle_aged_adult: {'Accuracy': 0.898955550157882, 'Precision': 0.5878155872667399, 'Recall': 0.9296875, 'F1 Score': 0.7202420981842637}
Group distribution:
 group
man      13154
woman    12380
Name: count, dtype: int64
Gender Results:
wo

In [21]:
def calculate_fairness_stats(privileged_total, privileged_checked, unprivileged_total, unprivileged_checked):
    unprivileged_unchecked = unprivileged_total - unprivileged_checked
    privileged_unchecked = privileged_total - privileged_checked

    favorable_outcome_given_unprivileged = unprivileged_unchecked / unprivileged_total if unprivileged_total > 0 else 0
    favorable_outcome_given_privileged = privileged_unchecked / privileged_total if privileged_total > 0 else 0

    spd = favorable_outcome_given_unprivileged - favorable_outcome_given_privileged
    di = (favorable_outcome_given_unprivileged / favorable_outcome_given_privileged 
          if favorable_outcome_given_privileged > 0 else float('inf'))

    return {
        "Disparate Impact": di,
        "Statistical Parity Difference": spd
    }

# Helper function to evaluate predicates
def evaluate_predicate(df, predicate):
    return df.loc[predicate(df)], df.loc[~predicate(df)]

# Function to calculate and display fairness statistics for predicates
def calculate_and_display_fairness(df, predicates, checked_col):
    for i, predicate in enumerate(predicates):
        # Apply the predicate
        subset, subset_complement = evaluate_predicate(df, predicate)
        
        # Calculate counts
        total_count = subset.shape[0]
        checked_count = subset[subset[checked_col] == True].shape[0]
        total_count_complement = subset_complement.shape[0]
        checked_count_complement = subset_complement[subset_complement[checked_col] == True].shape[0]

        # Calculate fairness statistics
        fairness_stats = calculate_fairness_stats(
            total_count_complement, checked_count_complement, 
            total_count, checked_count
        )

        # Display results
        print(f"Stats for Predicate {i+1}:")
        for stat, value in fairness_stats.items():
            print(f"  {stat}: {value:.4f}")
        print("-" * 50)

# Reusable predicate functions
def is_in_age_range(min_age, max_age):
    return lambda df: (df['persoon_leeftijd_bij_onderzoek'] >= min_age) & (df['persoon_leeftijd_bij_onderzoek'] < max_age)

def is_female():
    return lambda df: df['persoon_geslacht_vrouw'] == True

def is_male():
    return lambda df: df['persoon_geslacht_vrouw'] == False

def has_children():
    return lambda df: df['relatie_kind_heeft_kinderen'] == True

def no_children():
    return lambda df: df['relatie_kind_heeft_kinderen'] == False

def has_roommate():
    return lambda df: df['relatie_overig_kostendeler'] == True
    
def lacks_language_skills():
    return lambda df: df['persoonlijke_eigenschappen_taaleis_voldaan'] == False

def has_language_skills():
    return lambda df: df['persoonlijke_eigenschappen_taaleis_voldaan'] == True

def lives_in_district(district):
    return lambda df: df[f'adres_recentste_wijk_{district}'] == True

def financial_difficulties():
    return lambda df: df['belemmering_financiele_problemen'] == True

def combine_and(*predicates):
    return lambda df: np.logical_and.reduce([predicate(df) for predicate in predicates])

def combine_or(*predicates):
    return lambda df: np.logical_or.reduce([predicate(df) for predicate in predicates])



session = rt.InferenceSession("./received_folder/model_1.onnx")

# Prepare the input for the ONNX model
input_name = session.get_inputs()[0].name
X_test_np = X_test.to_numpy().astype('float32')

# Run predictions using the ONNX model
predictions = session.run(None, {input_name: X_test_np})[0]

# Add predictions to the dataframe
df = X_test.copy()  # Copy X_test to the new dataframe
df['checked'] = predictions  # Add predictions as the 'checked' column
# Define predicates
predicates = [
    is_in_age_range(0, 30),
    is_in_age_range(66, 120),
    combine_and(is_male(), is_in_age_range(20, 35), has_children()),
    combine_and(has_roommate(), financial_difficulties(), lacks_language_skills())
]


# Calculate and display fairness statistics
calculate_and_display_fairness(df, predicates, 'checked')

Stats for Predicate 1:
  Disparate Impact: 0.9025
  Statistical Parity Difference: -0.0774
--------------------------------------------------
Stats for Predicate 2:
  Disparate Impact: 0.8070
  Statistical Parity Difference: -0.1533
--------------------------------------------------
Stats for Predicate 3:
  Disparate Impact: 0.8709
  Statistical Parity Difference: -0.1023
--------------------------------------------------
Stats for Predicate 4:
  Disparate Impact: 0.5659
  Statistical Parity Difference: -0.3476
--------------------------------------------------
